# Modeling & Classification.




~~usar varios clusters, uno de 3grams, uno de nouns, uno de embeddings~~   
~~con esto buscar un clasificador multilabel (o simplemente agregarlos como features del modelo que le fedee al clasificador)~~  

Des métodos para clusterizar:    
-primero hago cluster separados para Tags, 2-3grams, Keys    
    luego agrego estas nuevas features al embedding, con eso puedo entrenar el clasificador.          
  
-segundo usando topic modeling q todavia no investigue     


Orden:  
Crear Espacio,  
normalizarlo,  
reducirlo,  
hacerle kmeans,  
obtener los clusters,  
medir siluete, gap, inertia, y precision con mi database.  
Agregar la medicion al df de mediciones.  


## Setup

In [24]:
import pandas as pd
import numpy as np
#import fasttext
from patterns import emoji_pattern
from gensim.models import Word2Vec, KeyedVectors
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.feature_selection import VarianceThreshold
from sentence_transformers import SentenceTransformer, util


df = pd.read_csv("db/features.csv", index_col=0, converters={'Qtoks': pd.eval,'Qstp': pd.eval,'Qkeys': pd.eval})

df


,id,Q,Qclean,Qtoks,Qstp,Qkeys,ANS,Aclean,Atoks,Astp,Akeys,QA-keys
0,ojjgvy,"people that eat spicy food, why?",people that eat spicy food why,"[people, that, eat, spicy, food, why]","[that, why]","[people, eat, spicy, food]","I never liked spicy food, it just adds a pain ...",I never liked spicy food it just adds a pain e...,"['I', 'never', 'liked', 'spicy', 'food', 'it',...","['it', 'just', 'a', 'that', 'i']","['I', 'never', 'liked', 'spicy', 'food', 'add'...","['people', 'eat', 'spicy', 'food', 'I', 'never..."
1,bc60pk,"donut or doughnut, & where are you from?",donut or doughnut where are you from,"[donut, or, doughnut, where, are, you, from]","[or, where, are, you, from]","[donut, doughnut]","donut, southern usa",donut southern usa,"['donut', 'southern', 'usa']",[],"['donut', 'southern', 'usa']","['donut', 'doughnut', 'donut', 'southern', 'usa']"
2,289mmm,help finding a song from a video description?,help finding a song from a video description,"[help, finding, a, song, from, a, video, descr...","[a, from, a]","[help, finding, song, video, description]",/r/tipofmytongue,rtipofmytongue,['rtipofmytongue'],[],['rtipofmytongue'],"['help', 'finding', 'song', 'video', 'descript..."
3,rufpr/,"i was rapedno, we had sex",i was rapedno we had sex,"[i, wa, rapedno, we, had, sex]","[i, we, had]","[wa, rapedno, sex]","if its not a yes, its no.",if its not a yes its no,"['if', 'it', 'not', 'a', 'yes', 'it', 'no']","['if', 'it', 'not', 'a', 'it', 'no']",['yes'],"['wa', 'rapedno', 'sex', 'yes']"
4,24vtw3,jesus h. christ what's his middle name?,jesus h christ whats his middle name,"[jesus, h, christ, whats, his, middle, name]",[his],"[jesus, h, christ, whats, middle, name]",umm... Holy. As in Holy Christ. Isn't this com...,umm Holy As in Holy Christ Isnt this common kn...,"['umm', 'Holy', 'As', 'in', 'Holy', 'Christ', ...","['in', 'this']","['umm', 'Holy', 'As', 'Holy', 'Christ', 'Isnt'...","['jesus', 'h', 'christ', 'whats', 'middle', 'n..."
...,...,...,...,...,...,...,...,...,...,...,...,...
90149,oqcloa,¿what easy work is really difficult?,¿what easy work is really difficult,"[¿what, easy, work, is, really, difficult]","[¿what, is]","[¿, easy, work, really, difficult]",None by virtue of it being easy in the first p...,None by virtue of it being easy in the first p...,"['None', 'by', 'virtue', 'of', 'it', 'being', ...","['by', 'of', 'it', 'being', 'in', 'the']","['None', 'virtue', 'easy', 'first', 'place']","['¿', 'easy', 'work', 'really', 'difficult', '..."
90150,ooswef,daddy what does nsfw mean?,daddy what does nsfw mean,"[daddy, what, doe, nsfw, mean]",[what],"[daddy, doe, nsfw, mean]",not safe for work,not safe for work,"['not', 'safe', 'for', 'work']","['not', 'for']","['safe', 'work']","['daddy', 'doe', 'nsfw', 'mean', 'safe', 'work']"
90151,orgpc4,"quiet people, what are your strengths?",quiet people what are your strengths,"[quiet, people, what, are, your, strength]","[what, are, your]","[quiet, people, strength]","Observing, attention, memory,",Observing attention memory,"['Observing', 'attention', 'memory']",[],"['Observing', 'attention', 'memory']","['quiet', 'people', 'strength', 'Observing', '..."
90152,og7m7k,what is the best indian movie?,what is the best indian movie,"[what, is, the, best, indian, movie]","[what, is, the]","[best, indian, movie]",Why is this a quote?,Why is this a quote,"['Why', 'is', 'this', 'a', 'quote']","['is', 'this', 'a']","['Why', 'quote']","['best', 'indian', 'movie', 'Why', 'quote']"


## Modeling

podria ser capa de LSA? (a los embedings tmb?   )

In [25]:
Models = {}
gpu = False

### Tfidf Models

In [26]:
def tdidf_model(df, column, range=(2,3), min_df=20, threshold=1e-3, ngram=False):
    
    docs = df[column].fillna("")

    if ngram:
        tdidf = TfidfVectorizer(min_df=min_df, ngram_range=range)
    else:
        tdidf = TfidfVectorizer(min_df=min_df)

    #Create, Normalize and Reduce 
    model = tdidf.fit_transform(docs)
    model = normalize(model, axis=1, norm="max")
    model = VarianceThreshold(threshold=threshold).fit_transform(model)

    return np.array(model)

In [27]:

#Qkeys nonstop lemmas
df["Q-kstr"] =  df["Qkeys"].apply(lambda x: " ".join(a for a in x))
df["QA-kstr"] =  df["Qkeys"].apply(lambda x: " ".join(a for a in x))
df["Q-stpstr"] =  df["Qkeys"].apply(lambda x: " ".join(a for a in x))


Models["Q-keys"] = tdidf_model(df, column="Q-kstr", min_df=15)

#Qclean is without punct
Models["Q-ngrams"] = tdidf_model(df, column="Qclean", range=(2,3), min_df=15, ngram = True)

#Question+Answer nonstop lemmas
Models["QA-keys"] = tdidf_model(df, column="QA-kstr", min_df=15)

#i think it can help tu
Models["Q-stop"] = tdidf_model(df, column="Q-stpstr", min_df=1)

### Embeddings

https://kavita-ganesan.com/fasttext-vs-word2vec/

In [28]:

def w2v_doc_embedding(docs_toks):

    docs_model = []
    
    words_model = Word2Vec(docs_toks, size=300, workers=8, seed=0)

    for tokens in docs_toks:
        zero_vector = np.zeros(words_model.vector_size)
        vectors = []
        for token in tokens:
            if token in words_model.wv:
                try:
                    vectors.append(words_model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            docs_model.append(avg_vec)
        else:
            docs_model.append(zero_vector)
    return np.array(docs_model)

Models["w2v-Q"] = w2v_doc_embedding(list(df["Qkeys"]))
Models["w2v-QA"] = w2v_doc_embedding(list(df["QA-keys"]))


### Pretrained Embeddings

- SBERT , glove-6B-300
    * Q, QLess, QA, QAless  


In [29]:
if gpu:
    glove60 = KeyedVectors.load_word2vec_format("glove.6B.300d.txt", binary=False, no_header=True)
    def glove_doc_embedding(docs_toks, words_model):

        docs_model = []
        

        for tokens in docs_toks:
            zero_vector = np.zeros(words_model.vector_size)
            vectors = []
            for token in tokens:
                try:
                    vectors.append(words_model[token])
                except KeyError:
                    continue
            if vectors:
                vectors = np.asarray(vectors)
                avg_vec = vectors.mean(axis=0)
                docs_model.append(avg_vec)
            else:
                docs_model.append(zero_vector)
        return docs_model
        
    Models["glove-Q"] = glove_doc_embedding(list(df["Qtoks"]), glove60)


In [30]:
if gpu:
    model = SentenceTransformer('nreimers/MiniLM-L6-H384-uncased') #
    Models["SBERT"] = model.encode(list(df["Q"].astype('str')), batch_size=70, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True)


### Topic Modeling
https://github.com/ddangelov/Top2Vec
https://pythonrepo.com/repo/ddangelov-Top2Vec-python-natural-language-processing

In [31]:
from top2vec import Top2Vec
if gpu:
    Models["top2vec"] = Top2Vec(list(df["Q"].astype('str')), embedding_model='universal-sentence-encoder')

In [32]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

dictionary = Dictionary(df["Qstp"])
bow_corpus = [dictionary.doc2bow(doc) for doc in df["Qtoks"]]

Models["LDA-keys"] = LdaMulticore(list(), 
                                   num_topics = 15,
                                   id2word = dictionary,                                    
                                   passes = 80,
                                   workers = 8)

for idx, topic in Models["LDA-keys"].print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")
#df

Topic: 0 
Words: 0.007*"after" + 0.007*"them" + 0.007*"him" + 0.007*"having" + 0.007*"once" + 0.007*"s" + 0.007*"shouldn" + 0.007*"out" + 0.007*"through" + 0.007*"isn"


Topic: 1 
Words: 0.007*"her" + 0.007*"against" + 0.007*"on" + 0.007*"few" + 0.007*"hot…why" + 0.007*"not" + 0.007*"been" + 0.007*"we" + 0.007*"he" + 0.007*"how"


Topic: 2 
Words: 0.008*"under" + 0.008*"very" + 0.008*"home…what" + 0.007*"those" + 0.007*"was…" + 0.007*"being" + 0.007*"any" + 0.007*"whom" + 0.007*"against" + 0.007*"disappointment—and"


Topic: 3 
Words: 0.008*"it" + 0.008*"about" + 0.008*"all" + 0.007*"as" + 0.007*"your" + 0.007*"whom" + 0.007*"after" + 0.007*"over" + 0.007*"too" + 0.007*"o"


Topic: 4 
Words: 0.008*"gossip—why" + 0.008*"a…" + 0.007*"aren" + 0.007*"again" + 0.007*"than" + 0.007*"we…" + 0.007*"shouldn" + 0.007*"won" + 0.007*"in" + 0.007*"after"


Topic: 5 
Words: 0.007*"be" + 0.007*"if" + 0.007*"should" + 0.007*"yourself" + 0.007*"have" + 0.007*"too" + 0.007*"home…what" + 0.007*"did" + 0.

## Clustering



Solo voy a probar con KMeans por cuestion de cantidad de datos y recursos, lo que me permite

https://scikit-learn.org/stable/modules/clustering.html
https://towardsdatascience.com/10-tips-for-choosing-the-optimal-number-of-clusters-277e93d72d92  
https://towardsdatascience.com/k-means-clustering-and-the-gap-statistics-4c5d414acd29  
https://hdbscan.readthedocs.io/en/latest/comparing_clustering_algorithms.html   s


In [33]:
import faiss


class FaissKMeans:
    def __init__(self, n_clus=10, n_init=10, max_iter=50):
        self.n_clusters = n_clus
        self.n_init = n_init
        self.max_iter = max_iter
        self.kmeans = None
        self.cluster_centers_ = None
        self.inertia_ = None

    def fit(self, X, y):
        self.kmeans = faiss.Kmeans(d=X.shape[1],
                                   k=self.n_clusters,
                                   niter=self.max_iter,
                                   nredo=self.n_init)
        self.kmeans.train(X.astype(np.float32))
        self.cluster_centers_ = self.kmeans.centroids
        self.inertia_ = self.kmeans.obj[-1]
        #print(self.kmeans.obj)

    def predict(self, X):
        return self.kmeans.index.search(X.astype(np.float32), 1)[1]

### Evaluation

inertia, silhuete, own_metric

In [34]:
#Con este dataset el metodo de elbow no se puede utiizar porque el error crece linearmente
def get_elbow(X,ran=(8,20)):
    #from kneed import KneeLocator, DataGenerator

    distortions = []
    for i in range(1000,1003):
        fkm = FaissKMeans(n_clus=i)
        fkm.fit(X, [])
        distortions.append(fkm.inertia_)
    return distortions

In [35]:
from sklearn.cluster import KMeans

# n_clusters=30
# kmc = KMeans(n_clusters=n_clusters,  random_state=0)
# kmc.fit(X=Models["top2vec"])

# df["label1"] = kmc.labels_

#vemos los primeros clusters, al ser con ngrams  la forma en la que comienza la pregunta tiene mucho peso
def print_kmc_clus(n=10):
    import random
    q_clus = [[] for i in range(n_clusters)]

    for sentence_id, cluster_id in enumerate(kmc.labels_):
        q_clus[cluster_id].append(df.iloc[sentence_id].Q)

    for i, cluster in enumerate(q_clus):
        print(f"Cluster {i+1}, len: {len(cluster)}" )
        for tw in random.sample(cluster, n):
            print(tw)
        print("\n")
        
#print_kmc_clus()


In [36]:

#Compara la cantidad de aciertos con los df de test
def check_results():
    test_df = pd.read_csv("db/test_db.csv", index_col=0)
    true_df = test_df[test_df["2"]]
    false_df = test_df[test_df["2"] == False]

    n = len(true_df)
    m = len(false_df)
    same_c = 0
    for i in range(n):
        if int(df.loc[true_df.iloc[i][0]].label1) ==  int(df.loc[true_df.iloc[i][1]].label1):
            same_c += 1
    
    diff_c = 0
    for i in range(m):
        if int(df.loc[false_df.iloc[i][0]].label1) !=  int(df.loc[false_df.iloc[i][1]].label1):
            diff_c += 1
        
    return same_c/n, diff_c/m
        
#check_results()
    

## Classifying 
